In [1]:
#----------------------------
# Model: AxialCFNet
# Written by: Ruchika Raj   
# Contact: ruchikar23@iitk.ac.in 
#----------------------------

# Dependency
import time
import numpy as np
import scipy.io as sio
import h5py 

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.optim import Adam
from timm.models.layers import to_2tuple, trunc_normal_

import torchprofile
from fvcore.nn import FlopCountAnalysis
from fvcore.nn import parameter_count

from colorama import Fore, Style
from tqdm.notebook import tqdm_notebook

import warnings
warnings.filterwarnings('ignore')
print(Fore.GREEN + '- Done' + Style.RESET_ALL)

- Done


/home/ruchikar23/miniconda3/lib/python3.13/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [2]:
# Device
if torch.cuda.is_available():
    device=torch.device('cuda')
else:
    device=torch.device('cpu')
print(Fore.BLUE +'❄️ Compute Device:' + Style.RESET_ALL, end=' ')
print(Fore.GREEN + str(device) + Style.RESET_ALL)    

❄️ Compute Device: cuda


In [3]:
# Dataset Class
# Dataset Class
class CustomDataset(Dataset):
    def __init__(self, path:str):
        self.a = sio.loadmat(path)['HT']
        self.b = np.reshape(self.a, (int(self.a.shape[0]/2), 4, 32, 32))
    
    def __len__(self):
        return len(self.b)
    
    def __getitem__(self, idx):
        sample = self.b[idx]
        return torch.tensor(sample, dtype=torch.float32)
    
# Dataset Class 2
class CustomDataset2(Dataset):
    def __init__(self, path:str):
        self.a = sio.loadmat(path)['HT']
        self.b = np.reshape(self.a, (int(self.a.shape[0]/2), 4, 32, 32))
        self.c = self.b[:, 2:4, :, :]
    
    def __len__(self):
        return len(self.c)
    
    def __getitem__(self, idx):
        sample = self.c[idx]
        return torch.tensor(sample, dtype=torch.float32)   

In [4]:
# Dataset
train_data = CustomDataset('DATA_Htrainin.mat')
val_data = CustomDataset('DATA_Hvalin.mat')
test_data = CustomDataset('DATA_Htestin.mat')
train_data2 = CustomDataset2('DATA_Htrainout.mat')

# DataLoader
train_loader = DataLoader(train_data, batch_size=256, shuffle=True)
train_loader2 = DataLoader(train_data2, batch_size=256, shuffle=True)
val_loader = DataLoader(val_data, batch_size=5000, shuffle=False)
test_loader = DataLoader(test_data, batch_size=6000, shuffle=False)

In [5]:
import os
print(os.path.exists('DATA_Htrainin.mat'))  # should return True


True


In [7]:
# NMSE Calculation
def NMSE(sparse_pred, sparse_gt):
    """ 
    Input:  (B, C, H, W)
    Output: NMSE
    Args:
        sparse_pred: (B, C, H, W)
        sparse_gt:   (B, C, H, W)
    Returns:
        x (float): NMSE
    """
    # Convert numpy arrays to PyTorch tensors
    if isinstance(sparse_gt, np.ndarray):
        sparse_gt = torch.tensor(sparse_gt, dtype=torch.float32)                              # (B, C, H, W)
    if isinstance(sparse_pred, np.ndarray):
        sparse_pred = torch.tensor(sparse_pred, dtype=torch.float32)                          # (B, C, H, W)
        
    # Moving to GPU    
    sparse_gt = sparse_gt.to(device)                                                          # (B, C, H, W)
    sparse_pred = sparse_pred.to(device)                                                      # (B, C, H, W)
    
    with torch.no_grad():
        # De-centralize
        sparse_gt = sparse_gt - 0.5                                                           # (B, C, H, W)
        sparse_pred = sparse_pred - 0.5                                                       # (B, C, H, W)
        # Calculate the NMSE
        power_gt = sparse_gt[:, 0, :, :] ** 2 + sparse_gt[:, 1, :, :] ** 2                    # (B, H, W)
        difference = sparse_gt - sparse_pred                                                  # (B, C, H, W)
        mse = difference[:, 0, :, :] ** 2 + difference[:, 1, :, :] ** 2                       # (B, H, W)
        nmse = 10 * torch.log10((mse.sum(dim=(1, 2)) / power_gt.sum(dim=(1, 2))).mean())      # NMSE
        
        return float(nmse) 
           

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class EfficientPatchEmbedding(nn.Module):
    def __init__(self, in_channels=2, embed_dim=32, patch_size=2):
        super().__init__()
        # Handle cases where input channels might differ
        self.proj = nn.Conv2d(in_channels, embed_dim, 
                            kernel_size=patch_size, 
                            stride=patch_size)
        self.patch_size = patch_size
        
    def forward(self, x):
        # If input has more channels than expected, take first 'in_channels' channels
        if x.shape[1] > self.proj.in_channels:
            x = x[:, :self.proj.in_channels, :, :]
        x = self.proj(x)  # [B, C, H, W] -> [B, D, H/p, W/p]
        B, C, H, W = x.shape
        return x.flatten(2).transpose(1, 2)  # [B, N, D]

class EfficientPatchMerging(nn.Module):
    def __init__(self, input_resolution, dim, out_dim):
        super().__init__()
        self.resolution = input_resolution
        self.dim = dim
        self.out_dim = out_dim
        self.norm = nn.LayerNorm(4 * dim)
        self.reduction = nn.Linear(4 * dim, out_dim, bias=False)
        
    def forward(self, x):
        B, N, C = x.shape
        H, W = self.resolution
        
        # First ensure the sequence length matches the expected resolution
        if N != H * W:
            raise ValueError(f"Input sequence length {N} doesn't match resolution {H}x{W}={H*W}")
            
        x = x.view(B, H, W, C)
        
        # Unfold patches
        x0 = x[:, 0::2, 0::2, :]  # B H/2 W/2 C
        x1 = x[:, 1::2, 0::2, :]
        x2 = x[:, 0::2, 1::2, :]
        x3 = x[:, 1::2, 1::2, :]
        x = torch.cat([x0, x1, x2, x3], -1)  # B H/2 W/2 4*C
        
        x = x.view(B, -1, 4 * C)
        x = self.norm(x)
        return self.reduction(x)



class EfficientPatchDivision(nn.Module):
    def __init__(self, input_resolution, dim, out_dim):
        super().__init__()
        self.resolution = input_resolution
        self.dim = dim
        self.out_dim = out_dim
        self.expand = nn.Linear(dim, 4 * out_dim, bias=False)
        self.norm = nn.LayerNorm(out_dim)
        
    def forward(self, x):
        B, N, C = x.shape
        H, W = self.resolution
        x = self.expand(x)
        x = x.view(B, H, W, 2, 2, self.out_dim)
        x = x.permute(0, 1, 3, 2, 4, 5).reshape(B, 2*H, 2*W, self.out_dim)
        x = x.view(B, -1, self.out_dim)
        return self.norm(x)

class WindowAxialAttention(nn.Module):
    def __init__(self, dim, num_heads=4, window_size=4, axis='h'):
        super().__init__()
        self.dim = dim
        self.num_heads = num_heads
        self.window_size = window_size
        self.axis = axis
        self.scale = (dim // num_heads) ** -0.5
        
        self.qkv = nn.Linear(dim, dim * 3, bias=False)
        self.proj = nn.Linear(dim, dim)
        
        # Relative position bias table
        self.relative_position_bias_table = nn.Parameter(
            torch.zeros((2 * window_size - 1) * (2 * window_size - 1), num_heads)
        )
        
        # Generate relative position indices
        coords = torch.arange(window_size)
        relative_coords = coords[:, None] - coords[None, :]  # [ws, ws]
        relative_coords += window_size - 1  # Shift to start from 0
        relative_position_index = relative_coords + relative_coords * (2 * window_size - 1)
        self.register_buffer("relative_position_index", relative_position_index.flatten())
        
        nn.init.trunc_normal_(self.relative_position_bias_table, std=.02)
        
    def forward(self, x):
        B, N, C = x.shape
        H = W = int(N ** 0.5)  # Assuming input is a square grid, H=W
        
        # Generate q, k, v
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]  # [B, num_heads, N, head_dim]
        
        # Reshape for window processing
        if self.axis == 'h':
            # Reshape for row-wise attention
            q = q.view(B, self.num_heads, H, W, -1)
            k = k.view(B, self.num_heads, H, W, -1)
            v = v.view(B, self.num_heads, H, W, -1)
            
            # Split into windows
            q = q.reshape(B * H, self.num_heads, W, -1)
            k = k.reshape(B * H, self.num_heads, W, -1)
            v = v.reshape(B * H, self.num_heads, W, -1)
        else:
            # Reshape for column-wise attention
            q = q.view(B, self.num_heads, H, W, -1)
            k = k.view(B, self.num_heads, H, W, -1)
            v = v.view(B, self.num_heads, H, W, -1)
            
            # Split into windows
            q = q.permute(0, 3, 1, 2, 4).reshape(B * W, self.num_heads, H, -1)
            k = k.permute(0, 3, 1, 2, 4).reshape(B * W, self.num_heads, H, -1)
            v = v.permute(0, 3, 1, 2, 4).reshape(B * W, self.num_heads, H, -1)
        
        # Scaled dot-product attention
        attn = (q @ k.transpose(-2, -1)) * self.scale  # [B*H, num_heads, W, W] or [B*W, num_heads, H, H]
        
        # Add relative position bias, resizing it to match the attention tensor
        relative_position_bias = self.relative_position_bias_table[
            self.relative_position_index
        ].view(self.window_size, self.window_size, -1)
        
        relative_position_bias = relative_position_bias.permute(2, 0, 1).unsqueeze(0)  # [1, num_heads, window_size, window_size]
        
        # Ensure the bias has the same size as the attention tensor
        if attn.shape[-2] != relative_position_bias.shape[-2]:
            relative_position_bias = F.interpolate(
                relative_position_bias, size=(attn.shape[-2], attn.shape[-2]), mode='bilinear', align_corners=False
            )
        
        attn = attn + relative_position_bias
        
        attn = attn.softmax(dim=-1)
        out = (attn @ v)  # [B*H, num_heads, W, head_dim] or [B*W, num_heads, H, head_dim]
        
        # Reshape back
        if self.axis == 'h':
            out = out.view(B, H, self.num_heads, W, -1).permute(0, 2, 1, 3, 4).reshape(B, N, C)
        else:
            out = out.view(B, W, self.num_heads, H, -1).permute(0, 2, 3, 1, 4).reshape(B, N, C)
        
        return self.proj(out)


class DepthwiseMLP(nn.Module):
    def __init__(self, dim, expansion=2):
        super().__init__()
        hidden_dim = int(dim * expansion)
        self.fc1 = nn.Linear(dim, hidden_dim)
        self.dwconv = nn.Conv1d(hidden_dim, hidden_dim, 3, padding=1, groups=hidden_dim)
        self.act = nn.GELU()
        self.fc2 = nn.Linear(hidden_dim, dim)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.dwconv(x.transpose(1, 2)).transpose(1, 2)
        x = self.act(x)
        return self.fc2(x)

class EfficientAxialBlock(nn.Module):
    def __init__(self, dim, num_heads, window_size=4):
        super().__init__()
        self.norm1 = nn.LayerNorm(dim)
        self.attn_h = WindowAxialAttention(dim, num_heads, window_size, 'h')
        self.attn_w = WindowAxialAttention(dim, num_heads, window_size, 'w')
        self.norm2 = nn.LayerNorm(dim)
        self.mlp = DepthwiseMLP(dim)
        
    def forward(self, x):
        x = x + self.attn_h(self.norm1(x))
        x = x + self.attn_w(self.norm1(x))
        x = x + self.mlp(self.norm2(x))
        return x

class ChannelCompression(nn.Module):
    def __init__(self, in_dim, out_dim):
        super().__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim
        
        # For expanding from lower dimension to higher
        self.expand_fc = nn.Linear(in_dim, out_dim * 4)  # Expansion factor of 4
        self.act = nn.GELU()
        
        # For compressing from higher dimension to lower
        self.compress_conv1 = nn.Conv1d(out_dim * 4, out_dim * 2, 1)
        self.compress_conv2 = nn.Conv1d(out_dim * 2, out_dim, 1)

    def forward(self, x):
        # Handle both expansion and compression cases
        if x.dim() == 2:  # [B, D] coming from linear layer (expansion case)
            x = self.expand_fc(x)  # [B, D*4]
            x = self.act(x)
            return x.view(x.size(0), -1, self.out_dim)  # [B, L, out_dim]
        elif x.dim() == 3:  # [B, L, D] coming from sequence (compression case)
            x = x.transpose(1, 2)  # [B, D, L]
            x = self.act(self.compress_conv1(x))
            x = self.compress_conv2(x)
            return x.flatten(1)  # [B, D*L]
        else:
            raise ValueError(f"Input must be 2D or 3D tensor, got {x.dim()}D")


class Encoder(nn.Module):
    def __init__(self, tx_codeword, in_channels=2):
        super().__init__()
        # For 32x32 input with patch_size=2, we get 16x16 patches
        self.patch_embed = EfficientPatchEmbedding(in_channels=2, embed_dim=32, patch_size=2)
        self.patch_norm    = nn.LayerNorm(32) 

        self.stage1 = nn.Sequential(
            EfficientAxialBlock(dim=32, num_heads=2)
        )

        # After patch embedding (patch_size=2), resolution is 16x16
        self.merge = EfficientPatchMerging(input_resolution=(16, 16), dim=32, out_dim=64)

        self.stage2 = nn.Sequential(
            EfficientAxialBlock(dim=64, num_heads=4)
        )
        self.precompress_norm = nn.LayerNorm(64) 

        self.compress = nn.Sequential(
            nn.Linear(8 * 8 * 64, 512),
            nn.GELU(),
            nn.Linear(512, tx_codeword)
        )

    def forward(self, x):
        x = self.patch_embed(x)           # [B, 256, 32]
        x = self.patch_norm(x)            # NEW
        x = self.stage1(x)                # [B, 256, 32]

        x = self.merge(x)                 # [B, 64, 64]
        x = self.stage2(x)                # [B, 64, 64]

        x = self.precompress_norm(x)      # NEW
        x = x.flatten(1)                  # [B, 64*64]
        x = self.compress(x)              # [B, tx_codeword]
        return x


class Decoder(nn.Module):
    def __init__(self, rx_codeword):
        super().__init__()
        self.expand = nn.Sequential(
            nn.Linear(rx_codeword, 512),
            nn.GELU(),
            nn.Linear(512, 8*8*64),
            nn.Unflatten(1, (8, 8, 64)) ) # [B, 8, 8, 64]
        
        self.stage1 = nn.Sequential(
            EfficientAxialBlock(dim=64, num_heads=4)
        )
        
        self.division = EfficientPatchDivision(input_resolution=(8, 8), dim=64, out_dim=32)
        self.postdivision_norm = nn.LayerNorm(32) 
        
        self.stage2 = nn.Sequential(
            EfficientAxialBlock(dim=32, num_heads=2)
        )
        
        self.output = nn.Sequential(
            nn.Linear(32, 16),
            nn.GELU(),
            nn.Linear(16, 2)
        )

    def forward(self, x):
        x = self.expand(x)       # [B, 8, 8, 64]
        x = x.view(-1, 8*8, 64)  # [B, 64, 64]
        x = self.stage1(x)       # [B, 64, 64]
        x = self.division(x)   
        x = self.postdivision_norm(x)   # [B, 256, 32]
        x = self.stage2(x)       # [B, 256, 32]
        
        # Reshape to spatial
        B = x.shape[0]
        x = x.view(B, 16, 16, 32)
        x = self.output(x)       # [B, 16, 16, 2]
        return x.permute(0, 3, 1, 2)  # [B, 2, 16, 16]


class AxialCFNet(nn.Module):
    def __init__(self, tx_codeword=128, in_channels=2):
        super().__init__()
        self.encoder = Encoder(tx_codeword, in_channels=2)
        self.decoder = Decoder(tx_codeword)
        
    def forward(self, x):
        latent = self.encoder(x)
        return self.decoder(latent)

In [9]:
# Initialization
torch.manual_seed(2025)
model = AxialCFNet(tx_codeword=512).to(device)

# Loss
criterion = nn.MSELoss()

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, betas=(0.9, 0.999), eps=1e-8)

In [10]:
#-----------------------------
# Model Params and Flop Count
#-----------------------------

# Model Params Count
def ParamCount(model):
    params=sum(p.numel() for p in model.parameters() if p.requires_grad)
    return print(Fore.BLUE + '- ParamsCount:'+ Style.RESET_ALL,
                Fore.GREEN + f'{params/1000000:.2f}M' + Style.RESET_ALL)


# Model Flops Count
def FlopCount(model):
    test=torch.randn(1, 2, 32, 32).to(device)
    flops=torchprofile.profile_macs(model, test)
    return print(Fore.BLUE + '- FlopsCount:'+ Style.RESET_ALL,
                Fore.GREEN + f'{flops/1000000:.2f}M' + Style.RESET_ALL)


# Params and Flops Count
def ParamFlopCount(model):
    test=torch.randn(1, 2, 32, 32).to(device)
    flop_counter=FlopCountAnalysis(model, test)
    flops=flop_counter.total()
    params = parameter_count(model)['']
    time_start=time.time()
    test_hat=model(test)
    time_end=time.time()
    total_time=time_end-time_start
    return {'Params': f'{params/1000000:.2f}M', 'Flops':f'{flops/1000000:.2f}M', 'RunTime': f'{total_time:.3f}s'}


# Model Params Count
def ParamCountLayers(model):
    params=0
    for name, param in model.named_parameters():
        if param.requires_grad or param.requires_grad:
            params+=param.numel()
            print(f"Layer: {name} | Size: {param.size()} | Parameters: {param.numel()}")
    return  params/1000000

In [11]:
ParamCount(model)
FlopCount(model)

- ParamsCount: 4.87M
- FlopsCount: 20.18M


In [12]:
ParamFlopCount(model)

Unsupported operator aten::mul encountered 33 time(s)
Unsupported operator aten::add encountered 20 time(s)
Unsupported operator aten::softmax encountered 8 time(s)
Unsupported operator aten::gelu encountered 7 time(s)
Unsupported operator aten::unflatten encountered 1 time(s)


{'Params': '4.87M', 'Flops': '20.58M', 'RunTime': '0.007s'}

In [14]:
import torch.nn.functional as F
from tqdm import tqdm

# ----------------
# Model Training
# ----------------

num_epochs = 1000
train_losses = []
val_losses = []
nmse_score = []

for epoch in tqdm(range(num_epochs), desc='Model Training', colour="#0099ff", leave=True):
    # Model Training
    model.train()  # Ensure the model is in train mode
    train_loss = 0
    for data in tqdm(train_loader, 'Mini Batch Training', colour="#0099ff", leave=False):
        x_resized = data.to(device)  # Assuming x_resized is the input
        optimizer.zero_grad()
        
        # Forward pass through the model
        model_out = model(x_resized)
        
        # Resize x_resized to match model_out shape (B, 2, 16, 16)
        x_resized = F.interpolate(x_resized, size=(16, 16), mode='bilinear', align_corners=False)
        
        # Reduce channels from 4 to 2 to match model_out channels (if necessary)
        conv = nn.Conv2d(4, 2, kernel_size=1)  # Reduce channels from 4 to 2
        conv.to(device)  # Ensure convolution layer is on the same device
        x_resized = conv(x_resized)
        
        # Calculate loss (now model_out and x_resized have the same shape)
        loss = criterion(model_out, x_resized)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        # Accumulate the training loss
        train_loss += loss.item() * x_resized.size(0)

    if (epoch+1) % 1 == 0:
        # Model Validation
        model.eval()  # Switch model to evaluation mode
        val_loss = 0
        with torch.no_grad():
            for data in tqdm(val_loader, 'Validating The Model', colour="#0099ff", leave=False):
                x_resized = data.to(device)
                output = model(x_resized)
                x_resized = F.interpolate(x_resized, size=(16, 16), mode='bilinear', align_corners=False)
        
        # Reduce channels from 4 to 2 to match model_out channels (if necessary)
                conv = nn.Conv2d(4, 2, kernel_size=1)  # Reduce channels from 4 to 2
                conv.to(device)  # Ensure convolution layer is on the same device
                x_resized = conv(x_resized)
                loss = criterion(output, x_resized)
                val_loss += loss.item() * x_resized.size(0)

        # Model Testing
        model.eval()
        nmse_error = 0
        with torch.no_grad():
            for data in tqdm(test_loader, 'Calculating NMSE', colour="#0099ff", leave=False):
                x_resized = data.to(device)
                output = model(x_resized)
                x_resized = F.interpolate(x_resized, size=(16, 16), mode='bilinear', align_corners=False)
        
        # Reduce channels from 4 to 2 to match model_out channels (if necessary)
                conv = nn.Conv2d(4, 2, kernel_size=1)  # Reduce channels from 4 to 2
                conv.to(device)  # Ensure convolution layer is on the same device
                x_resized = conv(x_resized)
                nmse = NMSE(output, x_resized) 
                nmse_error += nmse   

        # Calculate average training and validation loss
        train_loss /= len(train_loader.dataset)
        train_losses.append(train_loss)
        
        val_loss /= len(val_loader.dataset)
        val_losses.append(val_loss)
        
        # NMSE Score
        avg_nmse = nmse_error / len(test_loader)
        nmse_score.append(avg_nmse)
        
        # Printing training details
        print(Fore.LIGHTBLUE_EX + f'- Epoch: {epoch+1}/{num_epochs}' + Style.RESET_ALL)
        print(Fore.RED + f'- Train Loss: {train_loss:.9f} | Validation Loss: {val_loss:.9f}' + Style.RESET_ALL, end=' ')
        print(Fore.GREEN + f'| NMSE: {avg_nmse:.7f}' + Style.RESET_ALL)

        # Saving checkpoint
        if avg_nmse <= min(nmse_score):
            params = {'epoch': epoch+1,
                      'model': model.state_dict(),
                      'optimizer': optimizer.state_dict()}
            
            if (epoch+1) <= 400:
                torch.save(params, f'model400.pth')
            else:
                torch.save(params, f'model1000.pth')

    # Saving training details            
    np.savetxt(f'nmse_scores.csv', np.array(nmse_score), delimiter=",")
    np.savetxt(f'train_losses.csv', np.array(train_losses), delimiter=",")
    np.savetxt(f'val_losses.csv', np.array(val_losses), delimiter=",")

Model Training:   0%|          | 1/1000 [00:30<8:35:59, 30.99s/it]

- Epoch: 1/1000
- Train Loss: 0.167972678 | Validation Loss: 0.100999850 | NMSE: -7.4961016


Model Training:   0%|          | 2/1000 [01:02<8:41:00, 31.32s/it]

- Epoch: 2/1000
- Train Loss: 0.185223687 | Validation Loss: 0.332194830 | NMSE: -11.3191667


Model Training:   0%|          | 3/1000 [01:33<8:36:23, 31.08s/it]

...

- Epoch: 998/1000
- Train Loss: 0.165240073 | Validation Loss: 0.102345981 | NMSE: -5.9195240


Model Training: 100%|█████████▉| 999/1000 [6:37:29<00:09,  9.68s/it]

- Epoch: 999/1000
- Train Loss: 0.166371595 | Validation Loss: 0.100795540 | NMSE: -3.1067173


Model Training: 100%|██████████| 1000/1000 [6:37:38<00:00, 23.86s/it]

- Epoch: 1000/1000
- Train Loss: 0.160662836 | Validation Loss: 0.370752359 | NMSE: 3.3324040


In [12]:
import torch
import torchvision

print("Torch:", torch.__version__)
print("TorchVision:", torchvision.__version__)


Torch: 2.5.1.post302
TorchVision: 0.20.1


In [27]:
from tqdm.notebook import tqdm
for i in tqdm(range(100)):
    pass

  0%|          | 0/100 [00:00<?, ?it/s]